<a href="https://colab.research.google.com/github/GriPet12/mercorAiDetect/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sentencepiece transformers --quiet

# Import

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification
from tqdm import tqdm

# Load Data

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

train['answer'] = train['answer'].astype(str)
train['topic'] = train['topic'].astype(str)
test['answer'] = test['answer'].astype(str)
test['topic'] = test['topic'].astype(str)

# Dataset Class

In [ ]:
class TextDataset(Dataset):
    def __init__(self, topics, answers, labels=None):
        self.texts = [t + " [SEP] " + a for t, a in zip(topics, answers)]
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# Parameters

In [ ]:
MODEL_NAME = "microsoft/deberta-v3-base"
OUTPUT_DIR = "./deberta_v3_fulltrain"
os.makedirs(OUTPUT_DIR, exist_ok=True)

MAX_LEN = 512
BATCH_SIZE = 8
EPOCHS = 50
LR = 5e-7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer

In [ ]:
tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)